# Datasets

A [Dataset](https://nexus-forge.readthedocs.io/en/latest/interaction.html#dataset) is a specialization of a [Resource](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource) that provides users with operations to handle files, record their provenance and describe them with metadata.

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

Note: DemoStore doesn't implement file operations yet. Use the BluBrainNexus store instead when creating a config file.

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource

In [4]:
from kgforge.specializations.resources import Dataset

In [5]:
import pandas as pd

## Creation with files

In [6]:
! ls -p ../../data | egrep -v /$

associations.tsv
my_data.xwz
my_data_derived.txt
persons.csv
tfidfvectorizer_model_schemaorg_linking


In [7]:
persons = Dataset(forge, name="Interesting Persons")

In [8]:
persons.add_files("../../data/persons.csv")

In [9]:
forge.register(persons)

<action> _register_one
<succeeded> True


In [10]:
forge.as_json(persons)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/16ff246a-50f5-4ce5-b6ca-d874bc1f3760',
 'type': 'Dataset',
 'hasPart': {'distribution': {'type': 'DataDownload',
   'atLocation': {'type': 'Location',
    'store': {'id': 'https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault'}},
   'contentSize': {'unitCode': 'bytes', 'value': 52},
   'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/1ef5cc2a-16ac-4105-80b2-8fd64fcd8f02',
   'digest': {'algorithm': 'SHA-256',
    'value': '1dacd765946963fda4949753659089c5f532714b418d30788bedddfec47a389f'},
   'encodingFormat': 'text/csv',
   'name': 'persons.csv'}},
 'name': 'Interesting Persons'}

In [11]:
associations = Dataset(forge, name="Associations data")

In [12]:
associations.add_files("../../data/associations.tsv")

In [13]:
associations.add_derivation(persons)

In [14]:
forge.register(associations)

<action> _register_one
<succeeded> True


In [15]:
forge.as_json(associations)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/76cbbcf7-deb9-4b84-8a3e-c0a284e03b9e',
 'type': 'Dataset',
 'derivation': {'type': 'Derivation',
  'entity': {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/16ff246a-50f5-4ce5-b6ca-d874bc1f3760?rev=1',
   'type': 'Dataset',
   'name': 'Interesting Persons'}},
 'hasPart': {'distribution': {'type': 'DataDownload',
   'atLocation': {'type': 'Location',
    'store': {'id': 'https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault'}},
   'contentSize': {'unitCode': 'bytes', 'value': 477},
   'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/76ca0b51-5a8d-46b7-8a91-29f211cb12bb',
   'digest': {'algorithm': 'SHA-256',
    'value': '789aa07948683fe036ac29811814a826b703b562f7d168eb70dee1fabde26859'},
   'encodingFormat': 'text/tab-separated-values',
   'name': 'associations.tsv'}},
 'name': 'Associations data'}

In [32]:
# The argument overwrite: bool can be provided to decide whether to overwrite (True) existing files with the same name or
# to create new ones (False) with their names suffixed with a timestamp.

# A cross_bucket argument can be provided to download data from the configured bucket (cross_bucket=False - the default value) 
# or from a bucket different than the configured one (cross_bucket=True). The configured store should support crossing buckets for this to work.
associations.download(path="./downloaded/", source="parts")

In [33]:
! ls -l ./downloaded

total 16
-rw-r--r--  1 mfsy  staff  477 Jan  6 16:39 associations.tsv
-rw-r--r--  1 mfsy  staff  477 Jan  6 16:42 associations.tsv.20220106164227


In [18]:
# ! rm -R ./downloaded/

## Creation with resources

In [22]:
distribution_1 = forge.attach("../../data/associations.tsv")

In [23]:
distribution_2 = forge.attach("../../data/persons.csv")

In [24]:
jane = Resource(type="Person", name="Jane Doe", distribution=distribution_1)

In [25]:
john = Resource(type="Person", name="John Smith", distribution=distribution_2)

In [26]:
persons = [jane, john]

In [27]:
forge.register(persons)

<count> 2
<action> _register_many
<succeeded> True


In [28]:
dataset = Dataset(forge, name="Interesting people")

In [29]:
dataset.add_parts(persons)

In [30]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [31]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/3a650ade-4cb7-4b53-82e3-b8880818b010',
 'type': 'Dataset',
 'hasPart': [{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/943b282b-c902-4860-a6f4-45e0940f4956?rev=1',
   'type': 'Person',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/4d5d38d4-2b79-4385-b5d1-da18bd4119cc'},
   'name': 'Jane Doe'},
  {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/6c0a27b5-ad35-4627-adee-5b8124215aff?rev=1',
   'type': 'Person',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/c47859af-218e-423c-a97f-076d16902360'},
   'name': 'John Smith'}],
 'name': 'Interesting people'}

In [34]:
dataset.download(path="./downloaded/", source="parts")

In [35]:
! ls -l ./downloaded

total 32
-rw-r--r--  1 mfsy  staff  477 Jan  6 16:39 associations.tsv
-rw-r--r--  1 mfsy  staff  477 Jan  6 16:42 associations.tsv.20220106164227
-rw-r--r--  1 mfsy  staff  477 Jan  6 16:42 associations.tsv.20220106164237
-rw-r--r--  1 mfsy  staff   52 Jan  6 16:42 persons.csv


In [90]:
# ! rm -R ./downloaded/

### Creation from resources

In [36]:
dataset = Dataset.from_resource(forge, [jane, john], store_metadata=True)
print(*dataset, sep="\n")

{
    id: https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/943b282b-c902-4860-a6f4-45e0940f4956
    type: Person
    distribution:
    {
        type: DataDownload
        atLocation:
        {
            type: Location
            store:
            {
                id: https://bluebrain.github.io/nexus/vocabulary/diskStorageDefault
            }
        }
        contentSize:
        {
            unitCode: bytes
            value: 477
        }
        contentUrl: https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/4d5d38d4-2b79-4385-b5d1-da18bd4119cc
        digest:
        {
            algorithm: SHA-256
            value: 789aa07948683fe036ac29811814a826b703b562f7d168eb70dee1fabde26859
        }
        encodingFormat: text/tab-separated-values
        name: associations.tsv
    }
    name: Jane Doe
}
{
    id: https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/6c0a27b5-ad35-4627-adee-5b8124215aff
    type: Person
    distribution:
  

## Creation from a dataframe

See notebook `07 DataFrame IO.ipynb` for details on conversions of instances of Resource from a Pandas DataFrame.

### basics

In [37]:
dataframe = pd.read_csv("../../data/persons.csv")

In [38]:
dataframe

,type,name
0,Person,Marie Curie
1,Person,Albert Einstein


In [39]:
persons = forge.from_dataframe(dataframe)

In [40]:
forge.register(persons)

<count> 2
<action> _register_many
<succeeded> True


In [41]:
dataset = Dataset(forge, name="Interesting people")

In [42]:
dataset.add_parts(persons)

In [43]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [44]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/03e7a974-e2b4-4d67-8db4-b37448ef3ac2',
 'type': 'Dataset',
 'hasPart': [{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/c30555ed-ba4e-4ec6-9ca9-7d99c54173cf?rev=1',
   'type': 'Person',
   'name': 'Marie Curie'},
  {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/81dc5691-db2e-4dea-ad9c-c2f58ed55a47?rev=1',
   'type': 'Person',
   'name': 'Albert Einstein'}],
 'name': 'Interesting people'}

### advanced

In [45]:
dataframe = pd.read_csv("../../data/associations.tsv", sep="\t")

In [46]:
dataframe

,id,name,type,agent__type,agent__name,agent__gender__id,agent__gender__type,agent__gender__label,distribution
0,(missing),Curie Association,Association,Person,Marie Curie,http://purl.obolibrary.org/obo/PATO_0000383,LabeledOntologyEntity,female,../../data/scientists-database/marie_curie.txt
1,(missing),Einstein Association,Association,Person,Albert Einstein,http://purl.obolibrary.org/obo/PATO_0000384,LabeledOntologyEntity,male,../../data/scientists-database/albert_einstein...


In [47]:
dataframe["distribution"] = dataframe["distribution"].map(lambda x: forge.attach(x))

In [48]:
associations = forge.from_dataframe(dataframe, na="(missing)", nesting="__")

In [49]:
print(*associations, sep="\n")

{
    type: Association
    agent:
    {
        type: Person
        gender:
        {
            id: http://purl.obolibrary.org/obo/PATO_0000383
            type: LabeledOntologyEntity
            label: female
        }
        name: Marie Curie
    }
    distribution: LazyAction(operation=Store.upload, args=['../../data/scientists-database/marie_curie.txt', None])
    name: Curie Association
}
{
    type: Association
    agent:
    {
        type: Person
        gender:
        {
            id: http://purl.obolibrary.org/obo/PATO_0000384
            type: LabeledOntologyEntity
            label: male
        }
        name: Albert Einstein
    }
    distribution: LazyAction(operation=Store.upload, args=['../../data/scientists-database/albert_einstein.txt', None])
    name: Einstein Association
}


In [50]:
forge.register(associations)

<count> 2
<action> _register_many
<succeeded> True


In [51]:
dataset = Dataset(forge, name="Interesting associations")

In [52]:
dataset.add_parts(associations)

In [53]:
forge.register(dataset)

<action> _register_one
<succeeded> True


In [54]:
forge.as_json(dataset)

{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/19bd480f-6036-4197-a710-8b2cd0777600',
 'type': 'Dataset',
 'hasPart': [{'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/1d21c77b-8384-48b2-8069-dbbbb82b8332?rev=1',
   'type': 'Association',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/7f84cf2b-a251-404a-b66c-ab80af7b24af'},
   'name': 'Curie Association'},
  {'id': 'https://sandbox.bluebrainnexus.io/v1/resources/github-users/mfsy/_/70dada05-46dd-4be4-b3a2-9d9bc1da6d07?rev=1',
   'type': 'Association',
   'distribution': {'contentUrl': 'https://sandbox.bluebrainnexus.io/v1/files/github-users/mfsy/95b275f9-0326-4651-95db-368362a61126'},
   'name': 'Einstein Association'}],
 'name': 'Interesting associations'}